In [1]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd

In [2]:
now = datetime.now()
search_time = now.strftime("%Y-%m-%d %H:%M")
search_time

'2020-02-11 21:06'

In [3]:
chrome_options = Options()  
chrome_options.add_argument("--headless")  
chrome_options.add_argument("--no-sandbox")

driver = webdriver.Chrome('chromedriver', options=chrome_options)

In [4]:
last_num_hours = 1
url = f'https://news.google.com/search?q=%22Nigeria%22%20%22disease%22%20%22mystery%22%20when%3A{last_num_hours}h&hl=en'
driver.get(url)

In [5]:
def extract_title_and_url(page_content):
    soup = BeautifulSoup(content, 'lxml')

    # find <a> elements inside <h3> elements
    selector = 'h3 > a '
    titles = [item.text for  item in soup.select(selector)]
    hrefs = [item['href'] for  item in soup.select(selector)]
    
    selector = 'h3 ~ div ~ div > div > time '
    datetimes = [item['datetime'] for  item in soup.select(selector)]
    
    
    return pd.DataFrame({'title': titles, 'link': hrefs, "datetime": datetimes})

In [6]:
content = driver.page_source

new_df = extract_title_and_url(content)
new_df

,title,link,datetime


In [7]:
csv_file = 'data/google-news-links.csv'
df = pd.read_csv(csv_file)


In [8]:
df = df.append(new_df)

In [9]:
df.drop_duplicates(subset=['title'], inplace=True, keep='first')

,title,link,datetime


In [10]:
df.to_csv(csv_file, index=False)

In [11]:
df.shape

(0, 3)

### save the counts

In [12]:
count_csv_file = 'data/google-news-counts.csv'
df_counts = pd.read_csv(count_csv_file)

In [13]:
df_counts = df_counts.append({'search_time': search_time, 'total_count':df.shape[0]}, ignore_index=True)

In [14]:
df_counts.to_csv(count_csv_file, index=False)